In [59]:
import pandas as pd
import numpy as np
import networkx as nx

In [69]:
nodes = pd.read_csv("vienna_nodes.csv", sep=';')
coordinates = pd.read_csv("vienna_dist.csv", header = None, sep=' ')

In [85]:
nodes.tail()

,nodes,x,y
16075,16075,0.842712,0.287939
16076,16076,0.842689,0.287915
16077,16077,0.842687,0.287919
16078,16078,0.841141,0.284234
16079,16079,0.841437,0.283093


In [86]:
nodes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16080 entries, 0 to 16079
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   nodes   16080 non-null  int64  
 1   x       16080 non-null  float64
 2   y       16080 non-null  float64
dtypes: float64(2), int64(1)
memory usage: 377.0 KB


In [70]:
coordinates.columns = ['coord1','coord2','dist']
coordinates.head()

,coord1,coord2,dist
0,0,1367,112.736160
1,0,10445,176.062978
2,1,12881,92.183050
3,1,2676,16.391515
4,2,2664,173.987554


In [71]:
coordinates.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 36424 entries, 0 to 36423
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   coord1  36424 non-null  int64  
 1   coord2  36424 non-null  int64  
 2   dist    36424 non-null  float64
dtypes: float64(1), int64(2)
memory usage: 853.8 KB


In [74]:
graph = nx.DiGraph()

for _, row in coordinates.iterrows():
    graph.add_edge(row['coord1'], row['coord2'], weight=row['dist'])

In [75]:
shortest_path = nx.shortest_path(graph, 0,2)

In [76]:
nx.shortest_path(graph, 0,2)

[0, 10445.0, 2671.0, 2669.0, 10401.0, 2676.0, 2]

In [83]:
# Retrieve the actual distance between the nodes
actual_distance = sum(graph.get_edge_data(u, v)['weight'] for u, v in zip(shortest_path, shortest_path[1:]))

print(f"The actual distance between node {0} and node {2} is: {actual_distance}")

The actual distance between node 0 and node 2 is: 565.2582942952082


In [112]:
import torch
import random
import math
from torch.utils.data import Dataset

total_nodes = 16080

Lambda = 0.025 # request rate per min
dod = 0.5
horizon = 400
fDmean = 10
fDstd = 2.5
batch_size = 2

# static customer counts V = Lambda*horizon*(1-dod)/(dod+0.5)
V_static = int(Lambda*horizon*(1-dod)/(dod)+0.5)

# total customer count
V = int(Lambda*horizon/(dod) + 0.5)

gen = random.Random()
gen.seed() # uses the default system seed
unifDist = gen.random # uniform distribution
durDist = lambda: max(0.01, gen.gauss(fDmean, fDstd)) # normal distribution with fDmean and fDstd

In [111]:
nodes.head()

,nodes,x,y
0,0,0.841400,0.285538
1,1,0.841368,0.285464
2,2,0.841383,0.285455
3,3,0.841495,0.285523
4,4,0.841500,0.285506


In [134]:
random.randint(0,total_nodes-1)

9016

In [137]:
data = torch.randint(*(0,total_nodes-1), (2,10))

In [143]:
data

tensor([[10780, 11080,  6390,  8680, 14880,   163,  6310,  7430, 13921, 11645],
        [ 3417,  6824,  6631, 12260,  2328,  7088,  3127, 13745,  7779, 11747]])

In [146]:
data_vienna = pd.read_csv('vienna_cordinates.csv')
data_vienna.head()

,id,lats,lons,xcoords,ycoords
0,0,0.841400,0.285538,131.099240,9.996953
1,1,0.841368,0.285464,129.854728,9.200211
2,2,0.841383,0.285455,129.714953,9.576067
3,3,0.841495,0.285523,130.844197,12.366543
4,4,0.841500,0.285506,130.555200,12.510271


In [147]:
import torch
# Set random seed for reproducibility
torch.manual_seed(42)

In [166]:
# Randomly select 100 nodes from the DataFrame with replacement for each batch
batch_size = 2
num_nodes_per_batch = 10
num_batches = 2

batched_data = []

for _ in range(num_batches):
    selected_rows = torch.multinomial(torch.tensor(data_vienna.id, dtype=torch.float), num_samples=num_nodes_per_batch, replacement=True)
    print(selected_rows)
    selected_data = data_vienna.loc[selected_rows].reset_index(drop=True) 
    print(selected_data)
    batched_data.append(selected_data)

tensor([ 2847, 13261,  5751,  3814,  5597,  9223,  7959, 11012, 11201,  2125])
      id      lats      lons     xcoords    ycoords
0   2847  0.841241  0.284983  121.844750   6.025849
1  13261  0.841971  0.286685  150.197668  24.283084
2   5751  0.841627  0.284796  118.736276  15.676735
3   3814  0.841103  0.286038  139.419466   2.565909
4   5597  0.840694  0.284701  117.152938  -7.641169
5   9223  0.840164  0.283525   97.568584 -20.904668
6   7959  0.841750  0.286518  147.410605  18.753401
7  11012  0.840816  0.284098  107.109919  -4.612412
8  11201  0.840453  0.284864  119.862874 -13.678525
9   2125  0.842553  0.287506  163.883258  38.826782
tensor([11759, 12766, 15187, 11807, 14912,  4933, 11235, 13331,  7842,  8449])
      id      lats      lons     xcoords    ycoords
0  11759  0.840402  0.284952  121.330741 -14.947074
1  12766  0.841625  0.286323  144.169246  15.635152
2  15187  0.842325  0.286710  150.619521  33.136921
3  11807  0.840150  0.286360  144.788638 -21.239684
4  14912  

In [167]:
# Create PyTorch tensors for the batched data
batched_tensors = []
for batch_data in batched_data:
    id_tensor = torch.tensor(batch_data['id'].values, dtype=torch.long)
    coords_tensor = torch.tensor(batch_data[['lats', 'lons', 'xcoords', 'ycoords']].values, dtype=torch.float)
    batch_tensor = torch.cat((id_tensor.unsqueeze(1), coords_tensor), dim=1)
    batched_tensors.append(batch_tensor)

In [170]:
batched_tensors

[tensor([[ 2.8470e+03,  8.4124e-01,  2.8498e-01,  1.2184e+02,  6.0258e+00],
         [ 1.3261e+04,  8.4197e-01,  2.8668e-01,  1.5020e+02,  2.4283e+01],
         [ 5.7510e+03,  8.4163e-01,  2.8480e-01,  1.1874e+02,  1.5677e+01],
         [ 3.8140e+03,  8.4110e-01,  2.8604e-01,  1.3942e+02,  2.5659e+00],
         [ 5.5970e+03,  8.4069e-01,  2.8470e-01,  1.1715e+02, -7.6412e+00],
         [ 9.2230e+03,  8.4016e-01,  2.8353e-01,  9.7569e+01, -2.0905e+01],
         [ 7.9590e+03,  8.4175e-01,  2.8652e-01,  1.4741e+02,  1.8753e+01],
         [ 1.1012e+04,  8.4082e-01,  2.8410e-01,  1.0711e+02, -4.6124e+00],
         [ 1.1201e+04,  8.4045e-01,  2.8486e-01,  1.1986e+02, -1.3679e+01],
         [ 2.1250e+03,  8.4255e-01,  2.8751e-01,  1.6388e+02,  3.8827e+01]]),
 tensor([[ 1.1759e+04,  8.4040e-01,  2.8495e-01,  1.2133e+02, -1.4947e+01],
         [ 1.2766e+04,  8.4163e-01,  2.8632e-01,  1.4417e+02,  1.5635e+01],
         [ 1.5187e+04,  8.4233e-01,  2.8671e-01,  1.5062e+02,  3.3137e+01],
         [

In [171]:
20*1000/60

333.3333333333333

## DVRPSR Environment:


In [ ]:
import torch


class DVRPSR_Environment:
    vehicle_feature = 4  # vehicle coordinates (x_i, y_i), Vehicle_budget_capacity, travel_time
    customer_feature = 4

    # TODO: change pending cost for rewards

    def __init__(self, data, nodes=None, customer_mask=None, pending_cost=1, dynamic_reward=0.2):

        self.vehicle_count = data.vehicle_count
        self.vehicle_speed = data.vehicle_speed
        self.vehicle_budget = data.vehicle_time_budget

        self.nodes = data.nodes if nodes is None else nodes
        self.edges_index = data.edges_index
        self.edges_attributes = data.edges_attributes
        
        self.init_cust_mask = data.cust_mask if cust_mask is None else cust_mask

        self.minibatch, self.nodes_count, _ = self.nodes.size()
        self.pending_cost = pending_cost
        self.dynamic_reward = dynamic_reward

    def _update_vehicles(self, dest, tau=0):

        # calculate travel time
        # TODO: 1) in real world setting we need to calculate the distance of arc
        # If nodes i and j are directly connected by a road segment (i, j) ∈ A, then t(i,j)=t_ij;
        # otherwise, t(i,j)=t_ik1 +t_k1k2 +...+t_knj, where k1,...,kn ∈ V are the nodes along the
        # shortest path from node i to node j.
        #      2) calculate stating time for each vehicle $\tau $, currently is set to zero
        
        dist = torch.pairwise_distance(self.cur_veh[:, 0, :2], dest[:, 0, :2], keepdim=True)
        tt = dist / self.veh_speed

        dyn_cust = (dest[:, :, 3] > 0).float()

        # budget left while travelling to destination nodes
        budget = tau + tt + dest[:, :, 2]
        # print(budget, tau, tt, dest[:,:,2])

        # update vehicle features based on destination nodes
        self.cur_veh[:, :, :2] = dest[:, :, :2]
        self.cur_veh[:, :, 2] -= budget
        self.cur_veh[:, :, 3] += tt

        # update vehicles states
        self.vehicles = self.vehicles.scatter(1,
                                              self.cur_veh_idx[:, :, None].expand(-1, -1, self.veh_feature),
                                              self.cur_veh)

        return dist, dyn_cust

    def _update_done(self, cust_idx):
        self.veh_done.scatter_(1, self.cur_veh_idx, torch.logical_or((cust_idx == 0), (self.cur_veh[:, :, 2] <= 0)))
        # print(self.veh_done, cust_idx==0,self.cur_veh[:,:,2]<=0, (cust_idx==0) | (self.cur_veh[:,:,2]<=0))
        self.done = bool(self.veh_done.all())

    def _update_mask(self, cust_idx):

        self.new_customer = False
        self.served.scatter_(1, cust_idx, cust_idx > 0)

        # cost for a vehicle to go to customer and back to deport considering service duration
        cost = torch.pairwise_distance(self.cur_veh[:, :, :2], self.nodes[:, :, :2], keepdim=False) + \
               torch.pairwise_distance(self.nodes[:, None, 0, :2], self.nodes[:, :, :2], keepdim=False)
        cost = cost / self.veh_speed

        cost += self.nodes[:, :, 2]

        overtime_mask = self.cur_veh[:, :, 2] - cost
        overtime_mask = overtime_mask.unsqueeze(1)
        overtime = torch.zeros_like(self.mask).scatter_(1,
                                                        self.cur_veh_idx[:, :, None].expand(-1, -1, self.nodes_count),
                                                        overtime_mask < 0)

        self.mask = self.mask | self.served[:, None, :] | overtime | self.veh_done[:, :, None]
        self.mask[:, :, 0] = 0  # depot

    # updating current vehicle to find the next available vehicle
    def _update_cur_veh(self):
        avail = self.vehicles[:, :, 3].clone()
        avail[self.veh_done] = float('inf')

        self.cur_veh_idx = avail.argmin(1, keepdim=True)
        self.cur_veh = self.vehicles.gather(1, self.cur_veh_idx[:, :, None].expand(-1, -1, self.veh_feature))
        self.cur_veh_mask = self.mask.gather(1, self.cur_veh_idx[:, :, None].expand(-1, -1, self.nodes_count))

    def _update_hidden(self):

        time = self.cur_veh[:, :, 3].clone()

        if self.init_cust_mask is None:
            reveal_dyn_reqs = torch.logical_and((self.cust_mask), (self.nodes[:, :, 3] <= time))
        else:
            reveal_dyn_reqs = torch.logical_and((self.cust_mask ^ self.init_cust_mask), (self.nodes[:, :, 3] <= time))

        if reveal_dyn_reqs.any():
            self.new_customer = True
            self.cust_mask = self.cust_mask ^ reveal_dyn_reqs
            self.mask = self.mask ^ reveal_dyn_reqs[:, None, :].expand(-1, self.veh_count, -1)
            self.veh_done = torch.logical_and(self.veh_done, (reveal_dyn_reqs.any(1) ^ True).unsqueeze(1))
            self.vehicles[:, :, 3] = torch.max(self.vehicles[:, :, 3], time)
            self._update_cur_veh()

    def reset(self):
        # reset vehicle (minibatch*veh_count*veh_feature)
        self.vehicles = self.nodes.new_zeros((self.minibatch, self.veh_count, self.veh_feature))
        self.vehicles[:, :, :2] = self.nodes[:, :1, :2]
        self.vehicles[:, :, 2] = self.veh_budget

        # reset vehicle done
        self.veh_done = self.nodes.new_zeros((self.minibatch, self.veh_count), dtype=torch.bool)
        self.done = False

        # reset cust_mask
        self.cust_mask = self.nodes[:, :, 3] > 0
        if self.init_cust_mask is not None:
            self.cust_mask = self.cust_mask | self.init_cust_mask

        # reset new customers and served customer since now to zero (all false)
        self.new_customer = True
        self.served = torch.zeros_like(self.cust_mask)

        # reset mask (minibatch*veh_count*nodes)
        self.mask = self.cust_mask[:, None, :].repeat(1, self.veh_count, 1)

        # reset current vehicle idx, current vehicle, current vehicle mask
        self.cur_veh_idx = self.nodes.new_zeros((self.minibatch, 1), dtype=torch.int64)
        self.cur_veh = self.vehicles.gather(1, self.cur_veh_idx[:, :, None].expand(-1, -1, self.veh_feature))
        self.cur_veh_mask = self.mask.gather(1, self.cur_veh_idx[:, :, None].expand(-1, -1, self.nodes_count))

    def step(self, cust_idx):
        dest = self.nodes.gather(1, cust_idx[:, :, None].expand(-1, -1, self.cust_feature))
        dist, dyn_cust = self._update_vehicles(dest)

        #cust = (dest[:, :, 3] >= 0).float()

        self._update_done(cust_idx)
        self._update_mask(cust_idx)
        self._update_cur_veh()

        #reward = -dist * (1 - dyn_cust*self.dynamic_reward)
        reward = -dist + self.dynamic_reward*dyn_cust
        pending_static_cust = torch.logical_and((self.served ^ True), (self.nodes[:, :, 3] == 0)).float().sum(-1,keepdim=True) - 1
        reward -= self.pending_cost*pending_static_cust

        if self.done:

            if self.init_cust_mask is not None:
                self.served += self.init_cust_mask
            pending_cust = torch.logical_and((self.served ^ True), (self.nodes[:, :, 3] >= 0)).float().sum(-1, keepdim=True) - 1
            reward -= self.dynamic_reward*pending_cust

        self._update_hidden()

        return reward

    def state_dict(self, dest_dict=None):
        if dest_dict is None:
            dest_dict = {'vehicles': self.vehicles,
                         'veh_done': self.veh_done,
                         'served': self.served,
                         'mask': self.mask,
                         'cur_veh_idx': self.cur_veh_idx}

        else:
            dest_dict["vehicles"].copy_(self.vehicles)
            dest_dict["veh_done"].copy_(self.veh_done)
            dest_dict["served"].copy_(self.served)
            dest_dict["mask"].copy_(self.mask)
            dest_dict["cur_veh_idx"].copy_(self.cur_veh_idx)

        return dest_dict

    def load_state_dict(self, state_dict):
        self.vehicles.copy_(state_dict["vehicles"])
        self.veh_done.copy_(state_dict["veh_done"])
        self.served.copy_(state_dict["served"])
        self.mask.copy_(state_dict["mask"])
        self.cur_veh_idx.copy_(state_dict["cur_veh_idx"])

        self.cur_veh = self.vehicles.gather(1, self.cur_veh_idx[:, :, None].expand(-1, -1, self.veh_feature))
        self.cur_veh_mask = self.mask.gather(1, self.cur_veh_idx[:, :, None].expand(-1, -1, self.cust_feature))


